In [ ]:
# !pip install rdflib

# import sys
# !{sys.executable} -m pip install pyenchant

In [48]:

import rdflib
from rdflib.namespace import RDFS
from urllib.parse import urldefrag
from rdflib.term import BNode,Literal
import pandas as pd
from rdflib import Graph
from collections import defaultdict
import numpy as np
import random

In [3]:
#Code to extract triples from turtle or ttl file (ontology)
def get_triples(graph):
    def get_frag(node):
        if '#' in node:
            return urldefrag(node)[1]
        else:
            return node
    def get_english(node,graph):
      if node in renameURIs:
        return renameURIs[node]
      elif (node,RDFS.label,None) in graph:
        labelNode = [x[2] for x in graph.triples((node,RDFS.label,None))][0]
        if isinstance(labelNode,Literal):
          return labelNode.value
        else:
          return labelNode
      elif (isinstance(node,Literal)):

        return node.value
      elif '#' in node.toPython():
        return get_frag(node.toPython())


    def change_case(string):
        
        if string is not None:
            string = string.replace("-", "")
            string = string.replace(" ", "_")
            if string:
                res = [string[0].lower()]
                for c in string[1:]:
                    if c in ('ABCDEFGHIJKLMNOPQRSTUVWXYZ'):
                        res.append('_')
                        res.append(c.lower())
                    else:
                        res.append(c)

                return ''.join(res)
            return string


    triples = []
    for s, p, o in graph:
      #out.write("\t".join([s,p,o]) +"\n")
      if isinstance(s,BNode) or isinstance(p,BNode) or isinstance(o,BNode) or  (p in (prefixURI,RDFS.label,contributorURI,licenseURI,commentURI,noteURI,definitionURI,deprecatedURI,seeAlsoURI,scopeNoteURI)) or (p == typeURI and o in (classURI,namedIndividualURI)) or (s == versionInfoURI or p == versionInfoURI or o ==versionInfoURI):
        continue
      s_name = change_case(get_english(s,graph))
      p_name = change_case(get_english(p,graph))
      o_name = change_case(get_english(o,graph))


      if s_name is None or p_name is None or o_name is None:
        continue
      else:
        triples.append([s_name,p_name,o_name])
    return triples

def init_graph(filename):
    with open(filename, "r") as f:
        graph = rdflib.Graph()
        graph.parse(f,format="ttl")
        
        triples = get_triples(graph)
    
        return triples


In [ ]:

filename = "CRISPsubset.ttl"

tri1 = init_graph(filename)

triples = list(set(tuple(sorted(sub)) for sub in tri1))

df = pd.DataFrame(tri)
df.to_csv("combined.tsv",index=False,header=False,sep="\t")

In [ ]:
#Second script to extract triples from ttl files (ontology)
from rdflib import Graph
graph = Graph()
graph.parse('./doce.ttl', format='ttl')

s_list = []
p_list = []
o_list = []
for s, p, o in graph:
    s = s.split("/")[-1]
    s = s.split("#")[-1]
    s_list.append(s)
    
    p = p.split("/")[-1]
    p = p.split("#")[-1]
    p_list.append(p)
    
    o = o.split("/")[-1]
    o = o.split("#")[-1]
    o_list.append(o)

print(len(s_list))


new_s_list = []
for index,item in enumerate(s_list):
    new_s_list.append(change_case(item))

new_p_list = []
for index,item in enumerate(p_list):
    new_p_list.append(change_case(item))


new_o_list = []
for index,item in enumerate(o_list):
    new_o_list.append(change_case(item))


with open("train.txt", "w") as train:
    for index, item in enumerate(new_s_list[0:640]):
        line = new_s_list[index] + "\t" + new_p_list[index] + "\t" + new_o_list[index] + "\n"
        
        train.write(line) 

with open("test.txt", "w") as test:
    index = 640 
    for item in new_s_list[640:]:
        line = new_s_list[index] + "\t" + new_p_list[index] + "\t" + new_o_list[index] + "\n"
        index +=1
        
        test.write(line)
        
        
entities = set(new_s_list) 
for item in new_o_list:
    entities.add(item)
    
print(len(entities))

relations = set(new_p_list)
print(len(relations))


with open("entities.txt", "w") as file:
    for item in entities:
        line = item +  "\n"
        
        file.write(line) 

with open("relations.txt", "w") as file:
    for item in relations:
        line = item +  "\n"
        
        file.write(line) 

new_s_list1 = []
for index,item in enumerate(s_list):
    item = item.replace("-", " ")
    new_s_list1.append(item)

new_p_list1 = []
for index,item in enumerate(p_list):
    item = item.replace("-", " ")
    new_p_list1.append(item)


new_o_list1 = []
for index,item in enumerate(o_list):
    item = item.replace("-", " ")
    new_o_list1.append(item)
    

entities = set(new_s_list1) 
for item in new_o_list1:
    entities.add(item)

relations = set(new_p_list1)
    
    
with open("entitiestotext.txt", "w") as file:
    for item in entities:
        line = item +  "\n"
        file.write(line) 

with open("relationstotext.txt", "w") as file:
    for item in relations:
        line = item +  "\n"
        file.write(line) 

In [51]:
#Extracts relation instances that is predicates and entities form triples data
triples = []
def extract_rel_instances(data):
    
    p_list = set()
    e_list = set()

    for index,rows in data.iterrows():
        try:
            s_name = rows[0].replace(" ", "_")
            p_name = rows[1].replace(" ", "_")
            o_name = rows[2].replace(" ", "_")
            p_list.add(p_name)
            e_list.add(s_name)
            e_list.add(o_name)


            triples.append([s_name,p_name,o_name])
        except Exception as ex:
            continue

    return p_list, e_list, triples

In [53]:
#Shows relations and their correnponding number of occurances in the triple
def show_relations_instances(p_list,e_list, triples):
    count_list = []
    for predicate in p_list:
        count = 0
        for item in triples:
            if item[1] == predicate:
                count +=1 
        count_list.append(count)

    count_dict = []
    p_list = list(p_list)

    for index,item in enumerate(count_list):
        count_dict.append([p_list[index], item])

    print(len(count_dict))  

    pd.set_option('display.max_rows', None)
    df = pd.DataFrame(count_dict, columns=["Relationship", "Number of instances"])
    return df,count_dict
    


In [55]:
#Filters relations with instances greater than a threshold amount. Eg: triples with relations occuring in more than 10 triples.
def filter_relations(count_dict, triples, threshold):
    new_rels = set()
    for key, value in count_dict:
        if value > threshold:
            new_rels.add(key)

    print(new_rels)
    newtriples = []
    new_ent = set()
    for s,p,o in triples:
        if p in new_rels:
            newtriples.append([s,p,o])
            new_ent.add(s)
            new_ent.add(o)
    return newtriples, new_ent, new_rels
    

In [56]:
#Saves data in a file
def save_data(filename, data):
    with open (filename,"w") as file:
        for item in data:
            file.write(item)
            file.write("\n")
            
def save_data_text(filename, data):
    with open (filename,"w") as file:
        for item in data:
            text = item + "\t" + item.replace("_", " ")
            file.write(text)
            file.write("\n")

In [57]:
#Splits dataset into train, test and dev, using a threshold value
def train_validate_test_split(df, train_percent=.6, validate_percent=.2, seed=None):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[perm[:train_end]]
    validate = df.iloc[perm[train_end:validate_end]]
    test = df.iloc[perm[validate_end:]]
    return train, validate, test


In [58]:
#Extracts entities from a data
def extract_entities(file):
    with open(file, "r") as f:
        entity = []
        data = f.readlines()
        for d in data:
            d=d.split("\n")[0]
            entity.append(d)
            
    return entity

#Extract the current triples from the data
def current_triples(data):
    curr = []
    for index,rows in data.iterrows():
        s = rows[0]
        p = rows[1]
        o = rows[2]
        curr.append([s,p,o])
    return curr
    
#Creates negative examples of data for triple classification task with entities present in the data
def create_neg_examples(data, entities):
    triples = []
    curr = current_triples(data)
    for index,rows in data.iterrows():
        s = rows[0]
        p = rows[1]
        o = rows[2]
        
        checkpoint = True
        while checkpoint:
            neg_o = random.choice(entities)
            if [s,p,neg_o] not in curr and [neg_o, p, o] not in curr:
                triples.append([s,p,o, 1])
                if random.choice(range(0,20000)) % 2 == 0:
                    triples.append([s,p,neg_o, -1])
                else:
                    triples.append([neg_o,p,o, -1])
                checkpoint = False
            
    return triples

#Creates negative examples for triple classificaiton task and saves in csv destination file.
def create_neg_eg_and_save(source, destination, entsource):
    entities = extract_entities(entsource)
    data = pd.read_csv(source, sep="\t",header=None)
    triples = create_neg_examples(data, entities)
    df = pd.DataFrame(triples)
    df.to_csv(destination,index=False,header=False,sep="\t")

 
  

In [ ]:
#Extract Triples from the predicates file extracted from SemMedDB

from langdetect import DetectorFactory, detect
from langdetect.lang_detect_exception import LangDetectException
DetectorFactory.seed = 0

def get_triples_umls(filename, req_list):
    triples = []
    data = pd.read_csv(filename, sep=",")
    for index,rows in data.iterrows():
        one = str(rows[0])
        two = str(rows[1])
        three = str(rows[2])

        try:
            if detect(one) == "en" and detect(two) == "en" and detect(three) == "en":
                s_name = one.lower().replace(" ", "_")
                p_name = two.lower().replace(" ", "_")
                o_name = three.lower().replace(" ", "_")
                triples.append([s_name,p_name,o_name])

        except LangDetectException:
            continue
    return triples

import re

def replace_and_return_string(string):
    replacing_strings=[" ", "__"]
    string = re.sub(r"^\s+|\s+$", "", string)
    string = re.sub('[^a-zA-Z0-9 \n\.]', '_', string)
    string = string.replace(".","")
    for chars in replacing_strings:
        string = string.replace(chars, "_")
    
    return string

    
def get_triples_umls1(filename, req_list):
    triples = []
    data = pd.read_csv(filename, sep="\t")
    for index,rows in data.iterrows():
        one = str(rows[0])
        two = str(rows[1])
        three = str(rows[2])
        s_name = replace_and_return_string(one.lower())
        p_name = replace_and_return_string(two.lower())
        o_name = replace_and_return_string(three.lower())
        triples.append([s_name,p_name,o_name])
    return triples

triples1 = get_triples_umls1("<FileName>", [])

df3 = pd.DataFrame(triples1)
df3.to_csv("<TriplesFile>.tsv",index=False,header=False,sep="\t")

data1 = pd.read_csv("<TriplesFile>.tsv", sep="\t")

p_list, e_list, triples = extract_rel_instances(data1)
new_ent = e_list
new_rels = p_list

print(len(triples))
df,count_dict = show_relations_instances(p_list, e_list, triples1)
print(df)

In [60]:
#Saves formatted entities and relations with their corresponding texts.
save_data("<DataFolder>/entities.txt", new_ent)
save_data("<DataFolder>/relations.txt", new_rels)

save_data_text("<DataFolder>/entity2text.txt", new_ent)
save_data_text("<DataFolder>/relation2text.txt", new_rels)

In [ ]:
#Scripts For Relation Prediction Task

In [61]:
#Creates a relation dictionary to combine triples with that relation instance
def combine_relation_instances(p_list,e_list, triples):
    reldict = {}
    for predicate in p_list:
        itemslist = []
        for item in triples:
            if item[1] == predicate:
                itemslist.append(item)  
                
        reldict[predicate] = itemslist
    return reldict
    

In [ ]:
#Combines the relation instances, splits triples for each realtion into train, test and dev such that the datasets cover all types of relations.

relations = combine_relation_instances(p_list, e_list, triples1)

count =  0
for key,value in relations.items():
    pd.set_option('display.max_rows', None)
    df = pd.DataFrame(value)
    
    train, validate, test= train_validate_test_split(df, train_percent=.8, validate_percent=.1)
    train.to_csv("<DataFolder>/for-rel/train.tsv",index=False,header=False,sep="\t", mode="a")
    validate.to_csv("<DataFolder>/for-rel/dev.tsv",index=False,header=False,sep="\t", mode="a")
    test.to_csv("<DataFolder>/for-rel/test.tsv",index=False,header=False,sep="\t", mode="a")
    count += len(value)

print(count)

In [ ]:
#Required for LINK and RELATION prediction. Shuffles data in the dataset. 

def shuffle_data_and_save(file):
    data = pd.read_csv(file, sep="\t")
    df = data.sample(frac=1)
    df.to_csv(file,index=False,header=False,sep="\t")

In [261]:
shuffle_data_and_save("<DataFolder>/for-rel/train.tsv") 
shuffle_data_and_save("<DataFolder>/for-rel/test.tsv") 
shuffle_data_and_save("<DataFolder>/for-rel/dev.tsv")

#Note: You need to copy the entities, entity2text, relations, and relation2text to the <DataFolder>/for-rel. 

In [ ]:
# If you wish to filter relations with count greater than a given threshold the following pieces of code are useful else jump directly to Link classification scripts.
#Filtering RELATIONS 
#STEP 1

new_triples, new_ent, new_rels = filter_relations(count_dict, triples1, 100) #Here threshold is given 100
print(len(new_rels))
df,count_dict = show_relations_instances(new_rels, new_ent, new_triples)
print(df)


df_umls = pd.DataFrame(new_triples)
df_umls.to_csv("<DataFolder>/filtered/final.tsv",index=False,header=False,sep="\t")

save_data("<DataFolder>/filtered/entities.txt", new_ent)
save_data("<DataFolder>/filtered/relations.txt", new_rels)

save_data_text("<DataFolder>/filtered/entity2text.txt", new_ent)
save_data_text("<DataFolder>/filtered/relation2text.txt", new_rels)


In [248]:
#STEP 2
#For Relation - Viewing subject| object - with each unique relation

data = pd.read_csv("<DataFolder>/final.tsv",header=None, sep="\t", low_memory=False)
p_list = set()
e_list = set()
dictKey = defaultdict(list)

for index,rows in data.iterrows():
    s = rows[0].lower()
    p = rows[1].lower()
    o = rows[2].lower()
    
    p_list.add(p)
    e_list.add(s)
    e_list.add(o)
    key = s+ "|" + o
    dictKey[key].append(p)
        

trip1 = []
maxV = 0
for key, value in dictKey.items():
    maxV = max(maxV,len(value))
    trip1.append([key, value, len(value)])


In [ ]:
#Saving the subject/object realtion data in a file.
df2 = pd.DataFrame(trip1)
df2.to_csv("<DataFolder>/filtered/so-r.tsv",index=False,header=["subject | object", "relations", "count"],sep="\t")
print(maxV)

In [ ]:
#STEP 3
#Extracting new triples from the filtered list
out_trip = defaultdict(list)
overall = []
count = 0 
for rows in trip1:
    if rows[2] > 1:
        s, o = rows[0].split("|")
        for p in rows[1]:
            trip = [s,p,o]
            if trip in new_triples: 
                count +=1
                overall.append(trip)
                out_trip[rows[0]].append(trip)
                new_triples.remove(trip)

print("Done", count)
print(len(new_triples), len(overall))

In [ ]:
#Creating a train, test and validation splits from the new list of triples filtered with relations threshold.
#STEP 4

out_list = []
for key, value in out_trip.items():
    out_list.append(value)
    
n = len(out_list)
id1 = int(n * 0.8)
id2 = int(n * 0.8 * 0.1) + int(n * 0.8)

train3 = []
test3 = []
validate3 = []
count = 0

for item in out_list:
    if count >=0 and count < id1:
        for val in item:
            train3.append(val)
    elif count >= id1 and count < id2:
        for val in item:
            test3.append(val)
    elif count >=id2 and count < n:
        for val in item:
            validate3.append(val)
    count +=1

In [ ]:
#Continue for relation filtering  - Part2
#STEP 5

relations = combine_relation_instances(new_rels, new_ent, new_triples)
count =  0

train3 = pd.DataFrame(train3)
validate3 = pd.DataFrame(validate3)
test3 = pd.DataFrame(test3)

for key,value in relations.items():
    pd.set_option('display.max_rows', None)
    df = pd.DataFrame(value)
    
    tr, v, te= train_validate_test_split(df, train_percent=.8, validate_percent=.1)
    count += len(value)
    
    train3 = pd.concat([train3, tr])
    validate3 = pd.concat([validate3, v])
    test3 = pd.concat([test3, te])

train3.to_csv("<DataFolder>/filtered/train.tsv",index=False,header=False,sep="\t", mode='a')
validate3.to_csv("<DataFolder>/filtered/dev.tsv",index=False,header=False,sep="\t", mode='a')
test3.to_csv("<DataFolder>/filtered/test.tsv",index=False,header=False,sep="\t", mode='a')

print(len(train3), len(test3), len(validate3))

In [272]:
#STEP - 6
shuffle_data_and_save("<DataFolder>/filtered/train.tsv") 
shuffle_data_and_save("<DataFolder>/filtered/test.tsv") 
shuffle_data_and_save("<DataFolder>/filtered/dev.tsv")


In [ ]:
#--------------------
#LINK PREDICTION 
#If you are not doing link prediction then skip the scripts to Triple Classification

In [313]:
#Scripts For Link Prediction
#STEP 1

#Creating dictionary of key (subject entity + relation) and objects occuring with the same key
def view_link_relation_instances(p_list,e_list, triples):
    reldict = {}
    for item in triples:
        reldict[item[0]+"|"+item[1]] = []
    for item in triples:
        reldict[item[0]+"|"+item[1]].append(item[2])
    
    return reldict

In [314]:
#STEP 2
reldict = view_link_relation_instances(p_list, e_list, triples1)
count_dict = []

for key, value in reldict.items():
    count_dict.append([key, value, len(value)]) 

df2 = pd.DataFrame(count_dict)
df2.to_csv("<DataFolder>/for-link/er.tsv",index=False,header=["subject | relation", "tail entity", "number of instances"],sep="\t")


done


In [ ]:
#STEP 3

new_trip = triples1.copy()
 
out_trip = defaultdict(list)
overall = []
count = 0 

for rows in count_dict:
    if rows[2] > 1:
        s, p = rows[0].split("|")
        for o in rows[1]:
            trip = [s,p,o]
            if trip in new_trip: 
                count +=1
                overall.append(trip)
                out_trip[rows[0]].append(trip)
                new_trip.remove(trip)

print(len(new_trip), len(overall))

In [ ]:
#STEP 4
out_list = []
for key, value in out_trip.items():
    out_list.append(value)
    
n = len(out_list)
id1 = int(n * 0.8)
id2 = int(n * 0.8 * 0.1) + int(n * 0.8)

train2 = []
test2 = []
validate2 = []
count = 0

for item in out_list:
    if count >=0 and count < id1:
        for val in item:
            train2.append(val)
    elif count >= id1 and count < id2:
        for val in item:
            test2.append(val)
    elif count >=id2 and count < n:
        for val in item:
            validate2.append(val)
    count +=1


print(len(train2), len(test2), len(validate2))
print(len(train2) + len(test2) + len(validate2))

In [317]:
def combine_link_relation_instances(p_list, e_list, triples):
    linkdict = {}
    for item in triples:
        linkdict[item[0]+"|"+item[1]] = []
    for item in triples:
        linkdict[item[0]+"|"+item[1]].append(item)
    return linkdict

In [ ]:

#STEP 5

train2 = pd.DataFrame(train2)
validate2 = pd.DataFrame(validate2)
test2 = pd.DataFrame(test2)

df = pd.DataFrame(new_trip)

tr, v, te= train_validate_test_split(df, train_percent=.7, validate_percent=.15)
count += len(value)


train2 = pd.concat([train2, tr])
validate2 = pd.concat([validate2, v])
test2 = pd.concat([test2, te])

train2.to_csv("<DataFolder>/for-link/train.tsv",index=False,header=False,sep="\t", mode="a")
validate2.to_csv("<DataFolder>/for-link/dev.tsv",index=False,header=False,sep="\t", mode="a")
test2.to_csv("<DataFolder>/for-link/test.tsv",index=False,header=False,sep="\t", mode="a")

print(len(train2), len(test2), len(validate2))

In [328]:
#STEP 6
shuffle_data_and_save("<DataFolder>/for-link/train.tsv") 
shuffle_data_and_save("<DataFolder>for-link/test.tsv") 
shuffle_data_and_save("<DataFolder>/for-link/dev.tsv") 

In [ ]:
#----------------
#TRIPLE CLASSIFICATION

In [19]:
#For Triple classification 

train3, validate3, test3 = train_validate_test_split(df3, train_percent=.8, validate_percent=.1)
            
train3.to_csv("<DataFolder>/for-triple/train.tsv",index=False,header=False,sep="\t")
validate3.to_csv("<DataFolder>/for-triple/dev1.tsv",index=False,header=False,sep="\t")
test3.to_csv("<DataFolder>/for-triple/test1.tsv",index=False,header=False,sep="\t")


create_neg_eg_and_save("<DataFolder>/for-triple/test1.tsv", 
                       "<DataFolder>/for-triple/test.tsv",
                      "<DataFolder>/for-triple/entities.txt")

create_neg_eg_and_save("<DataFolder>/for-triple/dev1.tsv", 
                       "<DataFolder>/for-triple/dev.tsv",
                      "<DataFolder>/for-triple/entities.txt")
    


In [ ]:
#---------------------------------

In [ ]:
#Final UMLS Dataset Cleaning - Removing neg relations, inverse relations and same subject-object 

from collections import defaultdict
data = pd.read_csv("<DataFolder>/combined.tsv",header=None, low_memory=False, sep="\t")

count = 0
count1 = 0
count2 = 0 
p_list = set()
e_list = set()
triples = []
dictKey = defaultdict(list)

for index,rows in data.iterrows():
    s = rows[0].lower()
    p = rows[1].lower()
    o = rows[2].lower()
    if s == o: 
        count +=1
    elif [o,p,s] in triples:
        count1 +=1
    elif p.startswith("neg_"):
        count2 +=1
#     elif [s,p,o] not in triples:
    else:
        triples.append([s,p,o])
        p_list.add(p)
        e_list.add(s)
        e_list.add(o)
        
        
        key = s+ "|" + o
        dictKey[key].append(p)
        
         
print(count, count1,count2, len(triples))
  
df,count_dict = show_relations_instances(p_list, e_list, triples)
print(df)  

df1 = pd.DataFrame(triples)
df1.to_csv("<DataFolder>/init.tsv",index=False,header=False,sep="\t")

In [ ]:

#Supplement code for ranking triples and combining data triples from SemRepTool 

In [ ]:
#For Ranking tool

#Converting tsv to txt file
data = pd.read_csv("<PATH>/trip.tsv",header=None, sep="\t", low_memory=False)
data.to_csv("<PATH>/sub_rel_obj.txt",header=None, sep=" ", index=False)


In [ ]:
#Generating frequency score/ranking for subject object 

from sklearn import preprocessing

df = pd.read_csv('./stats.txt', sep='<>| ', header=None, skiprows=1,
                 names=['sub', 'rel', 'obj', 'rank', 'stat', 'f012',
                        'f0', 'f1', 'f2', 'f01', 'f02', 'f12'], engine='python')
deg = pd.read_csv('<PATH>/concept_degree.txt', sep=' ', header=None, names=['cui', 'deg'])
cui2deg = dict(zip(deg['cui'], deg['deg']))

def lookup(cui):
    try:
        return cui2deg[cui]
    except Exception as ex:
        return 0

df['sub_deg'] = df.apply(lambda row: lookup(row['sub']), axis=1)
df['obj_deg'] = df.apply(lambda row: lookup(row['obj']), axis=1)


names = ['stat', 'sub_deg', 'obj_deg']
x = df[names].values
min_max_scaler = preprocessing.MinMaxScaler()
x_norm = min_max_scaler.fit_transform(x)
df_temp = pd.DataFrame(x_norm, columns=names, index = df.index)
df[['stat_norm', 'sub_deg_norm', 'obj_deg_norm']] = df_temp
df['score'] = df['stat_norm'] + df['sub_deg_norm'] + df['obj_deg_norm']

df.to_csv('<PATH>/sub_rel_obj_freq_score.tsv', sep='\t', header=None, columns=['sub', 'rel', 'obj', 'f012', 'score'],
          index=False, float_format='%.15f')

In [ ]:
#Extracting max frequency
data = pd.read_csv("<PATH>/concept_degree.txt",header=None, sep=" ", low_memory=False)

max_lf = 0
for index,rows in data.iterrows():
    max_lf = max(max_lf,rows[1])

print(max_lf)

In [ ]:
#Removing duplicate triples
data = pd.read_csv("<FILE>",header=None, sep="\t", low_memory=False)

triples = []
for index,rows in data.iterrows():
    s = rows[0].lower()
    p = rows[1].lower()
    o = rows[2].lower()
    
    if [s,p,o] not in triples:
        triples.append([s,p,o])

print(len(triples))   
df2 = pd.DataFrame(triples)
df2.to_csv("<PATH>/originalremovingduplicates.tsv",index=False,header=None,sep="\t")

In [ ]:
data = pd.read_csv("<PATH>/sub_rel_obj_freq_score.tsv",header=None, sep="\t", low_memory=False)

remaining_rank_file = []
remaining_or_trip_file = []

rank_trip = []
or_trip = []
for index,rows in data.iterrows():
    s = rows[0]
    p = rows[1]
    o = rows[2]
    f = rows[3]
    score = rows[4]
    if [s,p,o] in triples:
        rank_trip.append([s,p,o,f,score])
        or_trip.append([s,p,o])
    else:
        remaining_rank_file.append([s,p,o,f,score])

In [ ]:
print(len(remaining_rank_file))
df2 = pd.DataFrame(remaining_rank_file)
df2.to_csv("<PATH>/remaining_ranking_triples.tsv",index=False,header=None,sep="\t")

In [ ]:
for item in triples:
    if item not in or_trip:
        remaining_or_trip_file.append(item)
    
print(len(remaining_or_trip_file))

In [ ]:
print(len(or_trip))
df2 = pd.DataFrame(remaining_or_trip_file)
df2.to_csv("<PATH>/remaining_original_triples.tsv",index=False,header=None,sep="\t")

In [ ]:
#To check weather the triples data contains the required concepts in the new dataset or not.

data = pd.read_csv("<PATH>/rowi.csv",header=None, sep=",", low_memory=False)
termsList = []
termsList1 = []
for index,rows in data.iterrows():
    terms = rows[0].lower()
    present = rows[3].lower()
    
    if present == "y":
        terms = replace_and_return_string(terms.lower())
        termsList.append(terms)
    elif present == "n":
        terms = replace_and_return_string(terms.lower())
        termsList1.append(terms)
        
        
data = pd.read_csv("<DataFolder>/combined-removing-neg-relations.tsv", header=None, sep="\t")
count = count1 = count2 = count3 = 0
presentList = []
solist = []

npresentList = []
nsolist = []
for index,rows in data.iterrows():
    s = replace_and_return_string(rows[0].lower())
    p = replace_and_return_string(rows[1].lower())
    o = replace_and_return_string(rows[2].lower())
    
    if s in termsList and o in termsList:
        count +=1
        presentList.append([s,p,o])
    elif s in termsList or o in termsList:
        count1 +=1
        solist.append([s,p,o])
        
    if s in termsList1 and o in termsList1:
        count2 +=1
        npresentList.append([s,p,o])
    elif s in termsList1 or o in termsList1:
        count3 +=1
        nsolist.append([s,p,o])
          
        
print(count, count1, count2, count3)

In [ ]:
df2 = pd.DataFrame(solist)
df2.to_csv("<DataFolder>/init.csv",index=False,header=None,sep="\t")

df2 = pd.DataFrame(presentList)
df2.to_csv("/Users/ngautam/Desktop/Abstract-related/abs-sem-comb/withoutdup/subjectobject_CUIs.tsv",index=False,header=None,sep="\t")

In [ ]:
key = ["Enterococci in water",
                    "Fecal Coliform in water",
                    "E.coli in marine water",
                    "Fecal Indicator Bacteria in water",
                    "E coli in water",
                    "Water quality indicators",
                    "Enteric Viruses",
                    "Enteric infections from beaches",
                    "Gastrointestinal Illness and beaches",
                    "Beach water contamination",
                    "Beach outbreaks",
                    "Waterborne infections in divers",
                    "Marine water quality",
                    "Marine water point source pollution",
                    "Quantitative microbial risk assessment and water",
                    "Weather and waterborne illness",
                    "Non cholera vibrio outbreaks",
                    "Pfiesteria infection",
                    "Red tides and illness",
                    "Harmful algal blooms and illness",
                    "Coastal water pollution",
                    "Water salinity and illness",
                    "Water characteristics and illness",
                    "Tides and illness",
                    "Harmful algal bloom"]
prev_keywords = []
for k in key:
    k = k.lower()
    prev_keywords.append(k)
    
    if k in termsList:
        print(k)

df = pd.DataFrame()
df['Keywords not in abstract list']  = pd.Series(termsList)
df['Abstract Keywords List']  = pd.Series(prev_keywords)
df['CUIS covered in SemMedDB']  = pd.Series(termsList)
df['CUIS not covered in SemMedDB']  = pd.Series(termsList1)

df.to_csv("<PATH>/info.tsv",index=False,sep="\t")

In [ ]:
# For extracting triples from keywords abstracts

def extract_trip_from_abstract(filename):
    dataset = []
    garbagge = []
    fulldata = []
    with open(filename, "r") as file:
        data = file.readlines()
        for line in data:
            if ("|relation|") in line:
                texts = line.split("|")
                if len(texts)  > 10:
                    s = texts[3]
                    p = texts[8]
                    o = texts[10]
                    if [s,p,o] not in dataset:
                        if s != "" and p != "" and o != "":
                            dataset.append([s,p,o])
                    fulldata.append([s,p,o])
                else:
                    garbagge.append(texts)
                    
    return fulldata, dataset, garbagge


f, d, g = extract_trip_from_abstract("<PATH>/Abstract-related/fullkey.txt")
f1, d1,g1 = extract_trip_from_abstract("<PATH>/Abstract-related/nlmout.txt")
f2, d2,g2 = extract_trip_from_abstract("<PATH>/Abstract-related/algalout.txt")

dataset = d + d1 + d2
fulldataset = f + f1 + f2
garbagge = g + g1 + g2

df = pd.DataFrame(dataset)
df.to_csv("<PATH>/Abstract-related/abstract-triples/withoutduplicates.tsv",index=False,header=None,sep="\t")
   
df1 = pd.DataFrame(garbagge)
df1.to_csv("<PATH>/Abstract-related/abstract-triples/gar.tsv",index=False,header=None,sep="\t")

df2 = pd.DataFrame(fulldataset)
df2.to_csv("<PATH>/Abstract-related/abstract-triples/fulldata.tsv",index=False,header=None,sep="\t")



In [ ]:
#Combine abstract and semMeddb triples for stats

data1 = pd.read_csv("<PATH>/Abstract-related/abs-sem-comb/withoutdup/abs.tsv",header=None, sep="\t", low_memory=False)
data2 = pd.read_csv("<PATH>/Abstract-related/abs-sem-comb/withoutdup/sem.tsv",header=None, sep="\t", low_memory=False)

combD = []
for index,rows in data1.iterrows():
    s = rows[0]
    p = rows[1]
    o = rows[2]
    combD.append([s,p,o])
    
    
count = 0
for index,rows in data2.iterrows():
    s = rows[0]
    p = rows[1]
    o = rows[2]
    if [s,p,o] not in combD:
        combD.append([s,p,o])
    else:
        count +=1
        
print("Duplicate triples in both files:", count)
print(len(data1), len(data2), len(combD))
        
df = pd.DataFrame(combD)
df.to_csv("<PATH>/Abstract-related/abs-sem-comb/withoutdup/combined.tsv",index=False,header=None,sep="\t")


In [ ]:

import pandas
import os, glob

url_list = set()
rootdir = "<PATH>/Abstract-related/backup" +  "/*"

top = 0 
for filename in glob.iglob(rootdir, recursive=True):
    if os.path.isfile(filename):
#         print(filename)
        count = 0 
        pd = pandas.read_csv(filename, header = None)

        for row in pd.itertuples():
            key = row[2]
            data = row[4]
            url = row[3]
            if data:
                count +=1
                if url not in url_list:
                    url_list.add(url)
        
        top +=1
        print(top, key,count)

In [399]:
#Extracting sentences corresponding to entities from Abstracts - 1 

data = pd.read_csv("<PATH>/trip.tsv",header=None, sep="\t", low_memory=False)

final = []
for index,rows in data.iterrows():
    s = rows[0]
    p = rows[1]
    o = rows[2]
    final.append([s,p,o])
    
print(len(final))

def extract_sentence_from_abstract(filename):
    dataset = []
    tripset = []

    with open(filename, "r") as file:
        data = file.readlines()
        for index, line in enumerate(data):
            if index < len(data) - 2:
                
                nextLine = data[index+2]
                
                if ("|relation|") in nextLine and ("|relation|") not in line:
                    
                    texts = nextLine.split("|")
                    textid = texts[0]
                    
                    sentence = line.split(textid)
                    if len(sentence) > 1:
                        sen = sentence[1].split("\n")[0]
                        dataset.append(sen.lower())

    return dataset


d2 = extract_sentence_from_abstract("<PATH>/Abstract-related/algalout.txt")
d = extract_sentence_from_abstract("<PATH>/Abstract-related/fullkey.txt")
d1 = extract_sentence_from_abstract("<PATH>/Abstract-related/nlmout.txt")

dataset = d + d1 + d2
print(len(dataset))

75628
102287


In [ ]:
#Extracting sentences corresponding to entities from Abstracts -2 

data = pd.read_csv("<PATH>/Abstract-related/rowi.csv",header=None, sep=",", low_memory=False)
termsList = []
for index,rows in data.iterrows():
    terms = rows[0].lower()
    present = rows[3].lower()
    
    if present == "y":
        terms = terms.lower()
        termsList.append(terms)

print(len(termsList))

sen_present = []
for item in dataset:
    for terms in termsList:
        if terms in item:
            if item not in sen_present:
                sen_present.append(item)
            
print(len(sen_present))

df = pd.DataFrame(sen_present)
df.to_csv("<PATH>/Abstract-related/abs-sentences/sentences-abs.txt",index=False,header=None)
